In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

**LSTM**

In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

class GRUModel(nn.Module):
    def __init__(self, input_size):
        super(GRUModel, self).__init__()
        self.gru1 = nn.LSTM(input_size=input_size, hidden_size=250, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(250, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.gru1(x)
        
        if len(x.shape) == 3:
            x = self.fc(x[:, -1, :])  # If the input has three dimensions, get the last output
        elif len(x.shape) == 2:
            x = self.fc(x.unsqueeze(1)).squeeze(1)  # If the input has two dimensions, add and squeeze a dimension
        else:
            raise ValueError("Input tensor must have either two or three dimensions.")
        
        x = self.sigmoid(x)  # Adjust to have the same size as the target
        return x






# Input shape
input_shape = (80, 636228, 1)

# Create the PyTorch model
model = GRUModel(input_shape[1])
print(model)

file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.float32)  # Change dtype to float32 for binary classification
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.float32)  # Change dtype to float32 for binary classification
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False) 

criterion = nn.BCEWithLogitsLoss()  # Change loss function for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training, Validation, and Testing loops
num_epochs = 10

# ...

for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_correct_train = 0
    total_samples_train = 0

    for batch_X, batch_y in train_loader:
        outputs = model(batch_X).squeeze(dim=1)

        # Ensure the target tensor is in the range [0, 1]
        batch_y = batch_y.float()  # Convert to float
        #batch_y = batch_y.unsqueeze(1)  # Ensure it has the correct shape

        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predicted_train = (outputs > 0.5).float()
        total_correct_train += (predicted_train == batch_y).sum().item()
        total_samples_train += batch_y.size(0)

    accuracy_train = total_correct_train / total_samples_train

    # Validation loop
    model.eval()
    total_correct_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            outputs_val = model(batch_X_val).squeeze(dim=1)

            # Ensure the target tensor is in the range [0, 1]
            batch_y_val = batch_y_val.float()  # Convert to float
            batch_y_val = batch_y_val.unsqueeze(1)  # Ensure it has the correct shape

            predicted_val = (outputs_val > 0.5).float()
            total_correct_val += (predicted_val == batch_y_val).sum().item()
            total_samples_val += batch_y_val.size(0)

    accuracy_val = total_correct_val / total_samples_val

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, '
          f'Training Accuracy: {accuracy_train * 100:.2f}%, '
          f'Validation Accuracy: {accuracy_val * 100:.2f}%')




GRUModel(
  (gru1): LSTM(636228, 250, batch_first=True)
  (fc): Linear(in_features=250, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch [1/10], Loss: 0.5787402391433716, Training Accuracy: 50.00%, Validation Accuracy: 236.36%
Epoch [2/10], Loss: 0.5879000425338745, Training Accuracy: 62.50%, Validation Accuracy: 236.36%
Epoch [3/10], Loss: 0.7392970323562622, Training Accuracy: 55.00%, Validation Accuracy: 222.73%
Epoch [4/10], Loss: 0.7620722055435181, Training Accuracy: 51.25%, Validation Accuracy: 245.45%
Epoch [5/10], Loss: 0.7293870449066162, Training Accuracy: 53.75%, Validation Accuracy: 222.73%
Epoch [6/10], Loss: 0.6814113855361938, Training Accuracy: 51.25%, Validation Accuracy: 222.73%
Epoch [7/10], Loss: 0.7721406817436218, Training Accuracy: 51.25%, Validation Accuracy: 222.73%
Epoch [8/10], Loss: 0.6910580396652222, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [9/10], Loss: 0.6835445165634155, Training Accuracy: 51.25%, Validation Accuracy: 222.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

class GRUModel(nn.Module):
    def __init__(self, input_size):
        super(GRUModel, self).__init__()
        self.gru1 = nn.LSTM(input_size=input_size, hidden_size=250, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(250, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.gru1(x)
        
        if len(x.shape) == 3:
            x = self.fc(x[:, -1, :])  # If the input has three dimensions, get the last output
        elif len(x.shape) == 2:
            x = self.fc(x.unsqueeze(1)).squeeze(1)  # If the input has two dimensions, add and squeeze a dimension
        else:
            raise ValueError("Input tensor must have either two or three dimensions.")
        
        x = self.sigmoid(x)  # Adjust to have the same size as the target
        return x






# Input shape
input_shape = (80, 636228, 1)

# Create the PyTorch model
model = GRUModel(input_shape[1])
print(model)

file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.float32)  # Change dtype to float32 for binary classification
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.float32)  # Change dtype to float32 for binary classification
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False) 

criterion = nn.BCEWithLogitsLoss()  # Change loss function for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training, Validation, and Testing loops
num_epochs = 10

# ...

for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_correct_train = 0
    total_samples_train = 0

    for batch_X, batch_y in train_loader:
        outputs = model(batch_X).squeeze(dim=1)

        # Ensure the target tensor is in the range [0, 1]
        batch_y = batch_y.float()  # Convert to float
        #batch_y = batch_y.unsqueeze(1)  # Ensure it has the correct shape

        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predicted_train = (outputs > 0.5).float()
        total_correct_train += (predicted_train == batch_y).sum().item()
        total_samples_train += batch_y.size(0)

    accuracy_train = total_correct_train / total_samples_train

    # Validation loop
    model.eval()
    total_correct_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            outputs_val = model(batch_X_val).squeeze(dim=1)

            # Ensure the target tensor is in the range [0, 1]
            batch_y_val = batch_y_val.float()  # Convert to float
            batch_y_val = batch_y_val.unsqueeze(1)  # Ensure it has the correct shape

            predicted_val = (outputs_val > 0.5).float()
            total_correct_val += (predicted_val == batch_y_val).sum().item()
            total_samples_val += batch_y_val.size(0)

    accuracy_val = total_correct_val / total_samples_val

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, '
          f'Training Accuracy: {accuracy_train * 100:.2f}%, '
          f'Validation Accuracy: {accuracy_val * 100:.2f}%')

GRUModel(
  (gru1): LSTM(636228, 250, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=250, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x500 and 250x1)

**GRU**

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

class GRUModel(nn.Module):
    def __init__(self, input_size):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(input_size=input_size, hidden_size=250, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(250, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.gru1(x)
        
        if len(x.shape) == 3:
            x = self.fc(x[:, -1, :])  # If the input has three dimensions, get the last output
        elif len(x.shape) == 2:
            x = self.fc(x.unsqueeze(1)).squeeze(1)  # If the input has two dimensions, add and squeeze a dimension
        else:
            raise ValueError("Input tensor must have either two or three dimensions.")
        
        x = self.sigmoid(x)  # Adjust to have the same size as the target
        return x






# Input shape
input_shape = (80, 636228, 1)

# Create the PyTorch model
model = GRUModel(input_shape[1])
print(model)

file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.float32)  # Change dtype to float32 for binary classification
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.float32)  # Change dtype to float32 for binary classification
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False) 

criterion = nn.BCEWithLogitsLoss()  # Change loss function for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training, Validation, and Testing loops
num_epochs = 10

# ...

for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_correct_train = 0
    total_samples_train = 0

    for batch_X, batch_y in train_loader:
        outputs = model(batch_X).squeeze(dim=1)

        # Ensure the target tensor is in the range [0, 1]
        batch_y = batch_y.float()  # Convert to float
        #batch_y = batch_y.unsqueeze(1)  # Ensure it has the correct shape

        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predicted_train = (outputs > 0.5).float()
        total_correct_train += (predicted_train == batch_y).sum().item()
        total_samples_train += batch_y.size(0)

    accuracy_train = total_correct_train / total_samples_train

    # Validation loop
    model.eval()
    total_correct_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            outputs_val = model(batch_X_val).squeeze(dim=1)

            # Ensure the target tensor is in the range [0, 1]
            batch_y_val = batch_y_val.float()  # Convert to float
            batch_y_val = batch_y_val.unsqueeze(1)  # Ensure it has the correct shape

            predicted_val = (outputs_val > 0.5).float()
            total_correct_val += (predicted_val == batch_y_val).sum().item()
            total_samples_val += batch_y_val.size(0)

    accuracy_val = total_correct_val / total_samples_val

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, '
          f'Training Accuracy: {accuracy_train * 100:.2f}%, '
          f'Validation Accuracy: {accuracy_val * 100:.2f}%')






GRUModel(
  (gru1): GRU(636228, 250, batch_first=True)
  (fc): Linear(in_features=250, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch [1/10], Loss: 0.5585112571716309, Training Accuracy: 46.25%, Validation Accuracy: 268.18%
Epoch [2/10], Loss: 0.7615493535995483, Training Accuracy: 52.50%, Validation Accuracy: 222.73%
Epoch [3/10], Loss: 0.6699438095092773, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [4/10], Loss: 0.7853032350540161, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [5/10], Loss: 0.6887288093566895, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [6/10], Loss: 0.7137421369552612, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [7/10], Loss: 0.6767549514770508, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [8/10], Loss: 0.6922649145126343, Training Accuracy: 50.00%, Validation Accuracy: 222.73%
Epoch [9/10], Loss: 0.7265862226486206, Training Accuracy: 51.25%, Validation Accuracy: 222.7

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class LSTMModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=250, batch_first=True, bidirectional=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.sigmoid(x)
        return x

# Input shape
input_shape = (80, 636228, 1)
num_classes = 2

# Create the PyTorch model
model = LSTMModel(input_shape[1], 2)
print(model)

file_path = 'Padded_Training.npz'
npz_file = np.load(file_path)

# Access individual arrays
TrainF = npz_file['features']
TrainL = npz_file['labels']

X_train_tensor = torch.tensor(TrainF, dtype=torch.float32)
y_train_tensor = torch.tensor(TrainL, dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

# Validation data
val_file_path = 'Padded_Testing.npz'
val_npz_file = np.load(val_file_path)
ValF = val_npz_file['features']
ValL = val_npz_file['labels']
X_val_tensor = torch.tensor(ValF, dtype=torch.float32)
y_val_tensor = torch.tensor(ValL, dtype=torch.long)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training, Validation, and Testing loops
num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    model.train()
    total_correct_train = 0
    total_samples_train = 0

    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted_train = torch.max(outputs, 1)
        total_correct_train += (predicted_train == batch_y).sum().item()
        total_samples_train += batch_y.size(0)

    accuracy_train = total_correct_train / total_samples_train

    # Validation loop
    model.eval()
    total_correct_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for batch_X_val, batch_y_val in val_loader:
            outputs_val = model(batch_X_val)
            _, predicted_val = torch.max(outputs_val, 1)
            total_correct_val += (predicted_val == batch_y_val).sum().item()
            total_samples_val += batch_y_val.size(0)

    accuracy_val = total_correct_val / total_samples_val

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, '
          f'Training Accuracy: {accuracy_train * 100:.2f}%, '
          f'Validation Accuracy: {accuracy_val * 100:.2f}%')


LSTMModel(
  (lstm1): LSTM(636228, 250, batch_first=True)
  (sigmoid): Sigmoid()
)
Epoch [1/10], Loss: 5.507979393005371, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch [2/10], Loss: 5.508194923400879, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch [3/10], Loss: 5.506131172180176, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch [4/10], Loss: 5.509807109832764, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch [5/10], Loss: 5.506494998931885, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch [6/10], Loss: 5.48881721496582, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch [7/10], Loss: 5.486522197723389, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch [8/10], Loss: 5.494141101837158, Training Accuracy: 0.00%, Validation Accuracy: 0.00%
Epoch [9/10], Loss: 5.5052571296691895, Training Accuracy: 2.50%, Validation Accuracy: 0.00%
Epoch [10/10], Loss: 5.419958114624023, Training Accuracy: 5.00%, Validation Accuracy: 9.

**SVM**

In [34]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_model = SVC(kernel='rbf', C=10)

# Train the model on the training data
svm_model.fit(TrainF, TrainL)

# Make predictions on the test data
y_pred = svm_model.predict(ValF)

# Evaluate the accuracy of the model
accuracy = accuracy_score(ValL, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 68.18%


**Naive Bayes**

In [35]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nb_model = GaussianNB()

nb_model.fit(TrainF, TrainL)

y_pred = nb_model.predict(ValF)

# Evaluate the accuracy of the model
accuracy = accuracy_score(ValL, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 50.00%


**Random Forest**

In [36]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you already have TrainF, TrainL, ValF, and ValL

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=50)

# Train the model on the training data
rf_model.fit(TrainF, TrainL)

# Make predictions on the test data
y_pred_rf = rf_model.predict(ValF)

# Evaluate the accuracy of the Random Forest model
accuracy_rf = accuracy_score(ValL, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")

Random Forest Accuracy: 54.55%
